In [38]:
import json
import pandas as pd
from typing import TypedDict
from datetime import datetime
import numpy as np

In [46]:
raw_data_json_path = '/home/artem/Projects/Psycat/data.json'
with open(raw_data_json_path, 'r') as rf:
    data = json.load(rf)

len(data.keys())

33

In [47]:
data_f1 = {}

for key in data:
    if  data[key].get('gilbert_test_passed') == 'yes' \
    and data[key].get('stroop_test_passed') == 'yes' \
    and data[key].get('ishihara_test_passed') == 'yes': 
        data_f1[key] = data[key]


In [49]:
class GilbertSeries(TypedDict):
    ans_a: int
    mean_a: float
    std_a: float
    mean_r: float
    ans_r: float
    std_r: float
    mean_l: float
    ans_l: float
    std_l: float
    ground: str
    target: str
    figure: str


class StroopSeries(TypedDict):
    count_a: int
    mean_a: float
    std_a: float
    count_r: float
    mean_r: float
    std_r: float
    count_l: float
    mean_l: float
    std_l: float


class RowStructure(TypedDict):
    code: str
    name: str
    datetime: datetime
    gilbert_test_results: list[GilbertSeries]
    stroop_test_results: list[StroopSeries]

print(data_f1['069632:bold_wheat_elephant']['gilbert_test_results'][0]['tests'][0])

{'reaction_time': 549, 'answer': 'r', 'direction': 'r', 'position': 3, 'correctly': True}


In [86]:
colors = {
    '#929292': 'grey',
    '#000000': 'black',

    'color(display-p3 0.9111 0.2904 0.3331)': 'r',
    'color(display-p3 0.8886 0.2848 0.5066)': 'red-in-pink',
    'color(display-p3 0.907 0.3246 0.0816)': 'red-out-orange',

    'color(display-p3 0.8675 0.7351 0.0446)': 'y',
    'color(display-p3 0.9686 0.67 0.1156)': 'yellow-in-orange',
    'color(display-p3 0.7347 0.7951 0.2312)': 'yellow-out-green',

    'color(display-p3 0.0772 0.7146 0.7601)': 'b',
    'color(display-p3 0.2086 0.6784 0.8873)': 'blue-in-violet',
    'color(display-p3 0.2336 0.727 0.5989)': 'blue-out-green',

}

Обнаружено: респонденты чаще ошибаются влево

In [127]:
data_f2: list[RowStructure] = []

for key in data_f1:
    row = data_f1[key]

    f_row = {
        'code': row['code'],
        'name': row['uid'],
        'datetime': datetime.fromtimestamp(row['datetime']['seconds']).strftime("%Y-%m-%d %H:%M:%S")
    }

    series = row['gilbert_test_results'][0]

    probes_a = []
    for probe in series['tests'][1:]:
        if probe['correctly']:
            probes_a.append(probe['reaction_time'])

    reaction = np.mean(probes_a)

    f_row['reaction'] = reaction

    index = 1

    for series in row['gilbert_test_results'][1:]:

        probes_a = []
        probes_r = []
        probes_l = []

        for probe in series['tests'][1:]:
            if not probe['correctly']:
                continue
            reaction_time = probe['reaction_time']
            if reaction_time > 1000:
                continue

            # reaction_time -= reaction

            probes_a.append(reaction_time)
            if probe['direction'] == 'r':
                probes_r.append(reaction_time)
            else:
                probes_l.append(reaction_time)

        key = f"{colors[series['ground']]}_{series['figure']}_{'in' if 'in' in colors[series['target']] else 'out'}"

        # if len(probes_a) < 11:
        #     print(len(probes_a),len(probes_l),len(probes_r))

        f_row[f"{index}_{key}"] = len(probes_a)
        f_row[f"cL_{key}"] = len(probes_l)
        f_row[f"cR_{key}"] = len(probes_r)
        f_row[f"A_{key}"] = np.mean(probes_a)
        f_row[f"L_{key}"] = np.mean(probes_l)
        f_row[f"R_{key}"] = np.mean(probes_r)


        index += 1

    data_f2.append(f_row)

full_table = pd.DataFrame(data_f2)

In [108]:
# tests = data_f1['090088:carefree_alizarin_starfish']['gilbert_test_results'][2]['tests']
# pd.DataFrame(tests)


In [141]:
client = data_f2[9]
print('code', client['code'])
print('name', client['name'])
print('datetime', client['datetime'])
print('reaction', client['reaction'])


{'in': }


{'code': '332301',
 'name': 'easygoing_chestnut_antelope',
 'datetime': '2024-04-20 17:50:01',
 'reaction': 649.6666666666666,
 '1_r_square_in': 10,
 'cL_r_square_in': 4,
 'cR_r_square_in': 6,
 'A_r_square_in': 695.5,
 'L_r_square_in': 768.25,
 'R_r_square_in': 647.0,
 '2_r_square_out': 9,
 'cL_r_square_out': 3,
 'cR_r_square_out': 6,
 'A_r_square_out': 710.5555555555555,
 'L_r_square_out': 821.6666666666666,
 'R_r_square_out': 655.0,
 '3_r_heart_in': 11,
 'cL_r_heart_in': 5,
 'cR_r_heart_in': 6,
 'A_r_heart_in': 707.3636363636364,
 'L_r_heart_in': 894.2,
 'R_r_heart_in': 551.6666666666666,
 '4_r_heart_out': 8,
 'cL_r_heart_out': 3,
 'cR_r_heart_out': 5,
 'A_r_heart_out': 683.5,
 'L_r_heart_out': 780.6666666666666,
 'R_r_heart_out': 625.2,
 '5_y_square_in': 10,
 'cL_y_square_in': 4,
 'cR_y_square_in': 6,
 'A_y_square_in': 660.3,
 'L_y_square_in': 745.0,
 'R_y_square_in': 603.8333333333334,
 '6_y_square_out': 11,
 'cL_y_square_out': 5,
 'cR_y_square_out': 6,
 'A_y_square_out': 651.27272

In [122]:
# full_table

In [128]:
full_table.to_csv('table.csv', float_format="%.8f")

In [132]:
full_table.keys()

Index(['code', 'name', 'datetime', 'reaction', '1_r_square_in',
       'cL_r_square_in', 'cR_r_square_in', 'A_r_square_in', 'L_r_square_in',
       'R_r_square_in', '2_r_square_out', 'cL_r_square_out', 'cR_r_square_out',
       'A_r_square_out', 'L_r_square_out', 'R_r_square_out', '3_r_heart_in',
       'cL_r_heart_in', 'cR_r_heart_in', 'A_r_heart_in', 'L_r_heart_in',
       'R_r_heart_in', '4_r_heart_out', 'cL_r_heart_out', 'cR_r_heart_out',
       'A_r_heart_out', 'L_r_heart_out', 'R_r_heart_out', '5_y_square_in',
       'cL_y_square_in', 'cR_y_square_in', 'A_y_square_in', 'L_y_square_in',
       'R_y_square_in', '6_y_square_out', 'cL_y_square_out', 'cR_y_square_out',
       'A_y_square_out', 'L_y_square_out', 'R_y_square_out', '7_y_sun_in',
       'cL_y_sun_in', 'cR_y_sun_in', 'A_y_sun_in', 'L_y_sun_in', 'R_y_sun_in',
       '8_y_sun_out', 'cL_y_sun_out', 'cR_y_sun_out', 'A_y_sun_out',
       'L_y_sun_out', 'R_y_sun_out', '9_b_square_in', 'cL_b_square_in',
       'cR_b_square_in', '

In [139]:
full_table[['reaction', 'A_r_square_in', 'L_r_square_in',
       'R_r_square_in', '2_r_square_out', 'cL_r_square_out', 'cR_r_square_out',
       'A_r_square_out', 'L_r_square_out', 'R_r_square_out', '3_r_heart_in',
       'cL_r_heart_in', 'cR_r_heart_in', 'A_r_heart_in', 'L_r_heart_in',
       'R_r_heart_in', '4_r_heart_out', 'cL_r_heart_out', 'cR_r_heart_out',
       'A_r_heart_out', 'L_r_heart_out', 'R_r_heart_out', '5_y_square_in',
       'cL_y_square_in', 'cR_y_square_in', 'A_y_square_in', 'L_y_square_in',
       'R_y_square_in', '6_y_square_out', 'cL_y_square_out', 'cR_y_square_out',
       'A_y_square_out', 'L_y_square_out', 'R_y_square_out', '7_y_sun_in',
       'cL_y_sun_in', 'cR_y_sun_in', 'A_y_sun_in', 'L_y_sun_in', 'R_y_sun_in',
       '8_y_sun_out', 'cL_y_sun_out', 'cR_y_sun_out', 'A_y_sun_out',
       'L_y_sun_out', 'R_y_sun_out', '9_b_square_in', 'cL_b_square_in',
       'cR_b_square_in', 'A_b_square_in', 'L_b_square_in', 'R_b_square_in',
       '10_b_square_out', 'cL_b_square_out', 'cR_b_square_out',
       'A_b_square_out', 'L_b_square_out', 'R_b_square_out', '11_b_blob_in',
       'cL_b_blob_in', 'cR_b_blob_in', 'A_b_blob_in', 'L_b_blob_in',
       'R_b_blob_in', '12_b_blob_out', 'cL_b_blob_out', 'cR_b_blob_out',
       'A_b_blob_out', 'L_b_blob_out', 'R_b_blob_out']].mean().to_dict()

{'reaction': 495.8083333333333,
 'A_r_square_in': 541.4565151515151,
 'L_r_square_in': 546.7458333333333,
 'R_r_square_in': 537.1783333333334,
 '2_r_square_out': 11.45,
 'cL_r_square_out': 5.65,
 'cR_r_square_out': 5.8,
 'A_r_square_out': 548.818952020202,
 'L_r_square_out': 554.545,
 'R_r_square_out': 546.3141666666668,
 '3_r_heart_in': 11.45,
 'cL_r_heart_in': 5.55,
 'cR_r_heart_in': 5.9,
 'A_r_heart_in': 518.1394696969699,
 'L_r_heart_in': 530.005,
 'R_r_heart_in': 507.92999999999995,
 '4_r_heart_out': 11.6,
 'cL_r_heart_out': 5.65,
 'cR_r_heart_out': 5.95,
 'A_r_heart_out': 536.7473484848485,
 'L_r_heart_out': 555.8033333333333,
 'R_r_heart_out': 520.2683333333333,
 '5_y_square_in': 10.9,
 'cL_y_square_in': 5.3,
 'cR_y_square_in': 5.6,
 'A_y_square_in': 631.6871590909091,
 'L_y_square_in': 633.4675,
 'R_y_square_in': 632.18,
 '6_y_square_out': 11.65,
 'cL_y_square_out': 5.7,
 'cR_y_square_out': 5.95,
 'A_y_square_out': 530.9024999999999,
 'L_y_square_out': 535.2924999999999,
 'R_y_

В всех случаях пиктограммы, в среднем, распознаются лучше!

Межкатегориальные распознаются лучше, кроме красного

Почти всюду правый глаз быстрее